<a href="https://colab.research.google.com/github/EmanueleGendy/operator-bases/blob/branch1/BruteForceMinSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding Invariants

This code produces all possible invariants built out of three matrices: C, Xu=Yu^\dagger Yu and Xd=Yd^\dagger Yd, where Yu and Yd are Yukawa matrices and C is a proxy for a matrix (in flavor space) of wilson coefficients of a higher dimensional operator in SMEFT. The invariants are required to be linear in C, i.e. C only appears once, while they contain as many Xu and Xd as needed to get to the desired degree "degg" fixed at the beginning. 

After having computed all the invariants of a fixed degree, we then check if there are linear relations within them. To this end, we create a linear system containing them and check if the associated matrix has a non-trivial null vector

In [1]:
import numpy as np
import sympy as sym
from scipy.linalg import null_space

In [2]:
degg = 4 # we will build all possible invariants built as products of traces of Xu, Xd and C, with the constraint they are linear in C 

In [3]:
def invlist(mat1, mat2):
    """
    Generate a list of traces of products of matrices and their identities up to a certain combination length.
    
    Parameters:
    - mat1, mat2: NumPy arrays representing matrices.
    
    Returns:
    - A list of traces of various matrix products.
    """
    combinations = [
        [mat1, np.identity(3)], [mat2, np.identity(3)],
        [mat1, mat1], [mat1, mat2], [mat2, mat2],
        [mat1, mat1, mat1], [mat1, mat1, mat2], [mat1, mat2, mat2], [mat2, mat2, mat2],
        [mat1, mat1, mat2, mat2]
    ]
    return [np.trace(np.linalg.multi_dot(combo)) for combo in combinations]

def invlistm(mat1, mat2, c):
    """
    Similar to `invlist` but includes an additional constant matrix `c` in the combinations.
    
    Parameters:
    - mat1, mat2, c: NumPy arrays representing matrices.
    
    Returns:
    - A list of traces of various matrix products including `c`.
    """
    combinations = [
        [c, np.identity(3)], [c, mat1], [c, mat2], 
        # Add further combinations as needed
    ]
    # Add more combinations as per your specific needs
    return [np.trace(np.linalg.multi_dot(combo)) for combo in combinations]

def recu(degree, ll, pr, name, totlist, prlist, namelist):
    """
    Recursive function to generate all invariants up to a specified degree.
    
    Parameters:
    - degree: The degree of invariants to generate.
    - ll: A current list of invariants.
    - pr: The current product of invariants.
    - name: The current name of the invariant being processed.
    - totlist: Accumulator for all combinations found.
    - prlist: Products of invariants.
    - namelist: Names of invariants.
    
    Returns:
    - A list containing prlist and namelist after the recursion finishes.
    """
    for i, exp in enumerate(expvec):
        if degree - exp < 0 and degree == 0 and ll not in totlist:
            ll.sort(key=lambda x: x[1])
            prlist.append(pr)
            namelist.append(name)
            totlist.append(ll)
        elif degree - exp >= 0:
            recu(degree - exp, ll + [[exp, listvec[i]]], pr * randinvlist[i], name + [listvec[i]], totlist, prlist, namelist)
    if i == len(expvec) - 1:
        return [prlist, namelist]

In [4]:
# Example usage and variable initialization
expvec = [1, 1, 2, 2, 2, 3, 3, 3, 3, 4]
listvec = ["a1", "b1", "a2", "b2", "c2", "a3", "b3", "c3", "d3", "a4"] # these correspond to the 10 invariants you can build with Yukawa matrices
# see arXiv:0907.4763, while "expvec" contains their degree

expvecm=[1,2,2,3,3,3,3,4,4,4,4,5,5,5,5,5,5,6,6,6,6,7,7,7,7,7,7,7,7] # these correspond to the invariants linear in C you can build with 
# Yukawa matrices and one instance of C up to degree 7 (for now this has to be done by hand)
listvecm=['I'+str(i) for i in range(len(expvecm))] # this is just a list of names for the invariants above

In [5]:
# Generate random matrices of integers 
m1 = np.random.randint(10, size=(3, 3))
m2 = np.random.randint(10, size=(3, 3))
c = np.random.randint(10, size=(3, 3))

# Generate random invariant lists
randinvlist = invlist(m1, m2)
randinvlistm = invlistm(m1, m2, c)

# Process to find invariants
systname = []
for i in range(len(listvecm)):
    if degg - expvecm[i] >= 0:
        systname += [[listvecm[i]] + j for j in recu(degg - expvecm[i], [], 1, [], [], [], [])[1]]
        
print(systname)
print(len(systname))

[['I0', 'a1', 'a1', 'a1'], ['I0', 'a1', 'a1', 'b1'], ['I0', 'a1', 'b1', 'a1'], ['I0', 'a1', 'b1', 'b1'], ['I0', 'a1', 'a2'], ['I0', 'a1', 'b2'], ['I0', 'a1', 'c2'], ['I0', 'b1', 'a1', 'a1'], ['I0', 'b1', 'a1', 'b1'], ['I0', 'b1', 'b1', 'a1'], ['I0', 'b1', 'b1', 'b1'], ['I0', 'b1', 'a2'], ['I0', 'b1', 'b2'], ['I0', 'b1', 'c2'], ['I0', 'a2', 'a1'], ['I0', 'b2', 'a1'], ['I0', 'b2', 'b1'], ['I0', 'c2', 'a1'], ['I0', 'c2', 'b1'], ['I0', 'a3'], ['I0', 'b3'], ['I0', 'c3'], ['I0', 'd3'], ['I1', 'a1', 'a1'], ['I1', 'a1', 'b1'], ['I1', 'b1', 'a1'], ['I1', 'b1', 'b1'], ['I1', 'a2'], ['I1', 'b2'], ['I1', 'c2'], ['I2', 'a1', 'a1'], ['I2', 'a1', 'b1'], ['I2', 'b1', 'a1'], ['I2', 'b1', 'b1'], ['I2', 'a2'], ['I2', 'b2'], ['I2', 'c2'], ['I3', 'a1'], ['I3', 'b1'], ['I4', 'a1'], ['I4', 'b1'], ['I5', 'a1'], ['I5', 'b1'], ['I6', 'a1'], ['I6', 'b1'], ['I7'], ['I8'], ['I9'], ['I10']]
49


In [6]:
# now to find relations between the invariants, we numerically create a matrix associated to the linear system built with the invariants themselves
# and then look for its null space

mat = []
for _ in systname:
    randinvlist = invlist(np.random.randint(100, size=(3, 3)), np.random.randint(100, size=(3, 3)))
    randinvlistm = invlistm(np.random.randint(100, size=(3, 3)), np.random.randint(100, size=(3, 3)), np.random.randint(1000, size=(3, 3)))
    syst=[]
    for i, val in enumerate(randinvlistm):
        if degg - expvecm[i] >= 0:
            syst += [val * j for j in recu(degg - expvecm[i], [], 1, [], [], [], [])[0]]
    mat.append(syst)

In [7]:
ns = null_space(mat)
M = sym.Matrix(mat)
a = M.nullspace()

In [8]:
print(f"Number of invariants: {len(systname)}")
print(f"Symbolic nullspace: {np.shape(a)}")

Number of invariants: 49
Symbolic nullspace: (11, 37, 1)
